In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
IBMQ.save_account('7d51f5c5e5c7521015c58312ad6428df9fc7dcf957d3159dfba1f25c77afdbee41b8ae64d75b87285e51cd8d0b28c172a4a13c2491283414a1d489281fb5a2f1')
provider = IBMQ.load_account()

/tmp/ipykernel_21426/1247502325.py:11: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.save_account('7d51f5c5e5c7521015c58312ad6428df9fc7dcf957d3159dfba1f25c77afdbee41b8ae64d75b87285e51cd8d0b28c172a4a13c2491283414a1d489281fb5a2f1')
configrc.store_credentials:WARNING:2023-02-27 23:15:07,354: Credentials already present. Set overwrite=True to overwrite.


In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
driver_ana = PySCFDriver(
    atom="H 0 0 0; Be 0 0 1.334; H 0 0 2.668",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [3]:
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
problem_ana = ElectronicStructureProblem(
        driver_ana,
        [FreezeCoreTransformer(freeze_core=True)])#,
                               #remove_orbitals=[-3,-2])])

In [4]:
problem_ana = driver_ana.run()

In [5]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, QubitConverter

solver = GroundStateEigensolver(
    QubitConverter(ParityMapper()),
    NumPyMinimumEigensolverFactory(),
)

In [6]:
result = solver.solve(problem_ana)
ana_gs = result.total_energies[0]
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -18.966849762441
  - computed part:      -18.966849762441
~ Nuclear repulsion energy (Hartree): 3.37181881021
> Total ground state energy (Hartree): -15.595030952231
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 6.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  15.1253679]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  15.1253679]
    - computed part:      [0.0  0.0  15.1253679]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [7]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)

# Define Molecule
molecule = Molecule(
    # Coordinates in Angstrom
    geometry=[
        ["H", [0.0, 0.0, 0.0]],
        ["Be", [1.334, 0.0, 0.0]],
        ["H", [2.668, 0.0, 0.0]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF)

In [8]:
# Define Problem, Use freeze core approximation, remove orbitals.
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

In [16]:
problem = ElectronicStructureProblem(
    driver,
    [FreezeCoreTransformer(freeze_core=True, remove_orbitals=[2,3])])

second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
num_spin_orbitals = problem.num_spin_orbitals
num_particles = problem.num_particles

In [17]:
print(num_spin_orbitals)
print(num_particles)

8
(1, 1)


In [18]:
from qiskit.opflow import TwoQubitReduction
mapper = ParityMapper()#JordanWignerMapper()
converter = QubitConverter(mapper, two_qubit_reduction=True)
hamiltonian = second_q_ops['ElectronicEnergy']
qubit_op = converter.convert(hamiltonian)
reducer = TwoQubitReduction(num_particles)
qubit_op = reducer.convert(qubit_op)
num_particles = num_particles
num_spin_orbitals = num_spin_orbitals -2

In [19]:
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import ExcitationPreserving
from qiskit.circuit import Parameter

var = HartreeFock(num_spin_orbitals//2, num_particles, converter)
var2 = HartreeFock(num_spin_orbitals//2, num_particles, converter)
var2.append( UCCSD(num_spin_orbitals//2, num_particles, converter), [i for i in range(num_spin_orbitals)])

In [20]:
'''for i in range(6):
    var.rx(Parameter('ax_' + str(i)), i)

var.append(ExcitationPreserving(2,reps=0, parameter_prefix='a'), [0,3])
var.append(ExcitationPreserving(2,reps=0, parameter_prefix='b'), [1,4])
var.append(ExcitationPreserving(2,reps=0, parameter_prefix='c'), [2,5])'''

"for i in range(6):\n    var.rx(Parameter('ax_' + str(i)), i)\n\nvar.append(ExcitationPreserving(2,reps=0, parameter_prefix='a'), [0,3])\nvar.append(ExcitationPreserving(2,reps=0, parameter_prefix='b'), [1,4])\nvar.append(ExcitationPreserving(2,reps=0, parameter_prefix='c'), [2,5])"

In [21]:
var.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘
q_3: ─────
          
q_4: ─────
          
q_5: ─────

In [22]:
from qiskit_aer.noise import NoiseModel
#from qiskit import CouplingMap
from qiskit.providers.aer import AerSimulator
backend = provider.get_backend('ibmq_jakarta')
backend_sim = AerSimulator.from_backend(backend)
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = backend.configuration().basis_gates

In [23]:
from qiskit.algorithms import MinimumEigensolver, VQEResult

# Define a custome VQE class to orchestra the ansatz, classical optimizers, 
# initial point, callback, and final result
class CustomVQE(MinimumEigensolver):
    
    def __init__(self, estimator, circuit, optimizer, callback=None):
        self._estimator = estimator
        self._circuit = circuit
        self._optimizer = optimizer
        self._callback = callback
        
    def compute_minimum_eigenvalue(self, operators, aux_operators=None):
                
        # Define objective function to classically minimize over
        def objective(x):
            # Execute job with estimator primitive
            job = self._estimator.run([self._circuit], [operators], [x], noise_model = noise_model)
            # Get results from jobs
            est_result = job.result()
            # Get the measured energy value
            value = est_result.values[0]
            # Save result information using callback function
            if self._callback is not None:
                self._callback(value)
            return value
            
        # Select an initial point for the ansatzs' parameters
        x0 = np.pi/4 * np.random.rand(self._circuit.num_parameters)
        
        # Run optimization
        res = self._optimizer.minimize(objective, x0=x0)
        
        # Populate VQE result
        result = VQEResult()
        result.cost_function_evals = res.nfev
        result.eigenvalue = res.fun
        result.optimal_parameters = res.x
        return result

In [24]:
q_layout = [0,1,2,3,5,6]
#ansatz_opt = transpile(var, backend=Aer.get_backend('aer_simulator'), initial_layout=q_layout)
ansatz_opt = transpile(var, AerSimulator(noise_model=noise_model), initial_layout=q_layout, optimization_level =3)

In [25]:
from qiskit.primitives import Estimator
import time

In [26]:
exp_qubit_op = []

for i in range(len(qubit_op)):
    aux = qubit_op[i].primitive.to_list()
    #print(aux[0][0])
    #print(aux[0][1])
    aux1 = aux[0][0][:4]+'I'+aux[0][0][4:]
    #print(aux1)
    
    exp_qubit_op.append((aux1, aux[0][1]))
    
from qiskit.opflow.primitive_ops import PauliSumOp

auxP = PauliSumOp.from_list(exp_qubit_op)
    

In [27]:
from qiskit.algorithms.optimizers import SPSA, COBYLA

#optimizer = SPSA(maxiter=150)
optimizer = COBYLA(maxiter=100)

In [28]:
# Define instance of qiskit-terra's Estimator primitive
estimator = Estimator([ansatz_opt], [auxP])
# Setup VQE algorithm

# Define a simple callback function
intermediate_info = []
def callback(value):
        intermediate_info.append(value)

custom_vqe = CustomVQE(estimator, ansatz_opt, optimizer, callback=callback)

# Run the custom VQE function and monitor execution time
start = time.time()
res_sim = custom_vqe.compute_minimum_eigenvalue(auxP)
end = time.time()

print(res_sim)
print(f'execution time (s): {end - start:.2f}')

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 1,
    'eigenstate': None,
    'eigenvalue': -1.312141384641212,
    'optimal_circuit': None,
    'optimal_parameters': array([], dtype=float64),
    'optimal_point': None,
    'optimal_value': None,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': None}
execution time (s): 0.01


In [30]:
cmp_gs = problem.interpret(res_sim).total_energies[0].real
print(problem.interpret(res_sim))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -18.931657247401
  - computed part:      -1.312141384641
  - FreezeCoreTransformer extracted energy part: -17.619515862759
~ Nuclear repulsion energy (Hartree): 3.37181881021
> Total ground state energy (Hartree): -15.559838437191
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [15.1253679  0.0  0.0]
 


In [31]:
print(cmp_gs)
print(ana_gs)
print((ana_gs-cmp_gs)/ana_gs*100)

-15.559838437190795
-15.59503095223091
0.2256649258851352


In [32]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator as RuntimeEstimator, Options
QiskitRuntimeService.save_account(channel="ibm_quantum", token='7d51f5c5e5c7521015c58312ad6428df9fc7dcf957d3159dfba1f25c77afdbee41b8ae64d75b87285e51cd8d0b28c172a4a13c2491283414a1d489281fb5a2f1' ,overwrite=True)
optimizer = COBYLA(maxiter=5)

In [33]:
# Define a simple callback function
intermediate_info_run = []
def callback_run(value):
        intermediate_info_run.append(value)

In [34]:
import signal, time

from qiskit_ibm_runtime import Estimator, Session
from qiskit.providers import JobStatus

def timeout_handler(signum, frame):
    raise Exception('Iteration timed out')
    
class RetryEstimator(Estimator):
    """RuntimeRetryEstimator class.
    
    This class inherits from Qiskit IBM Runtime's Estimator and overwrites its run method such that it retries calling it
    a maximum of 'max_retries' consecutive times, if it encounters one of the following randomly occuring errors:
    
    * An Estimator error (in this case "Job.ERROR" is printed, and the job is cancelled automatically)
    * A timeout error where the job either remains running or completes but does not return anything, for a time larger 
      than 'timeout' (in this case the job is cancelled by the patch and "Job.CANCELLED" is printed)
    * A creation error, where the job fails to be created because connection is lost between the runtime server and the
      quantum computer (in this case "Failed to create job." is printed). If this error occurs, the patch connects the user
      to a new Session (to be handled with care! also, this will unfortunately put the next job in the queue). 
    """
    
    def __init__(self, *args, max_retries: int = 5, timeout: int = 3600, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.max_retries = max_retries
        self.timeout = timeout
        self.backend = super().session._backend
        signal.signal(signal.SIGALRM, timeout_handler)
    
    def run(self, circuits, observables, parameter_values, **kwargs):
        result = None
        for i in range(self.max_retries):
            try:
                job = super().run(circuits, observables, parameter_values, **kwargs)
                while job.status() in [JobStatus.INITIALIZING, JobStatus.QUEUED, JobStatus.VALIDATING]:
                    time.sleep(5) # Check every 5 seconds whether job status has changed
                signal.alarm(self.timeout) # Once job starts running, set timeout to 1 hour by default
                result = job.result()
                if result is not None:
                    signal.alarm(0) # reset timer
                    return job
            except Exception as e:
                print("\nSomething went wrong...")
                print(f"\n\nERROR MESSAGE:\n{e}\n\n")
                if 'job' in locals(): # Sometimes job fails to create
                    print(f"Job ID: {job.job_id}. Job status: {job.status()}.")
                    if job.status() not in [JobStatus.DONE, JobStatus.ERROR, JobStatus.CANCELLED]:
                        job.cancel()
                else:
                    print("Failed to create job.")
                print(f"Starting trial number {i+2}...\n")
                print(f"Creating new session...\n")
                signal.alarm(0) # reset timer
                super().session.close()
                self._session = Session(backend=self.backend)
        if result is None:
            raise RuntimeError(f"Program failed! Maximum number of retries ({self.max_retries}) exceeded")

In [35]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', token=<IBM Quantum API key>)

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='qhack-event/main/level-1',
)
backend = service.get_backend('ibm_nairobi')
print(backend)

<IBMBackend('ibm_nairobi')>


In [36]:
ansatz_opt = transpile(var, backend=backend, initial_layout=q_layout, optimization_level =3)

In [38]:
options = Options()
options.execution.shots = 1024
options.optimization_level = 3
options.resilience_level = 3

In [39]:
start = time.time()
with Session(service=service, backend=backend) as session:
    #  Prepare primitive
    print(options)
    rt_estimator = RetryEstimator(session=session, options=options)# RuntimeEstimator(session=session, options=options)
     # Set up algorithm
    custom_vqe = CustomVQE(rt_estimator, ansatz_opt, optimizer, callback=callback_run)
     # Run algorithm
    result = custom_vqe.compute_minimum_eigenvalue(auxP)
end = time.time()
print(f'execution time (s): {end - start:.2f}')

Options(optimization_level=3, resilience_level=3, max_execution_time=None, transpilation=TranspilationOptions(skip_transpilation=False, initial_layout=None, layout_method=None, routing_method=None, approximation_degree=None), resilience=ResilienceOptions(noise_amplifier='TwoQubitAmplifier', noise_factors=(1, 3, 5), extrapolator='LinearExtrapolator'), execution=ExecutionOptions(shots=1024, init_qubits=True), environment=EnvironmentOptions(log_level='WARNING', callback=None, job_tags=[]), simulator=SimulatorOptions(noise_model=None, seed_simulator=None, coupling_map=None, basis_gates=None))
execution time (s): 33364.85


In [40]:
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 1,
    'eigenstate': None,
    'eigenvalue': -1.4429022960174533,
    'optimal_circuit': None,
    'optimal_parameters': array([], dtype=float64),
    'optimal_point': None,
    'optimal_value': None,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': None}


In [42]:
res_gs = problem.interpret(result).total_energies[0].real
print(problem.interpret(result))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -19.062418158777
  - computed part:      -1.442902296017
  - FreezeCoreTransformer extracted energy part: -17.619515862759
~ Nuclear repulsion energy (Hartree): 3.37181881021
> Total ground state energy (Hartree): -15.690599348567
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [15.1253679  0.0  0.0]
 


In [45]:
print(res_gs)
print(ana_gs)
print(abs((ana_gs-res_gs)/ana_gs*100))

-15.690599348567037
-15.59503095223091
0.612813123801177
